In [6]:
import os
import glob
import re
from collections import Counter
import numpy as np
import pandas as pd

# Para o modelo HMM
from hmmlearn import hmm


def get_text_list_from_files(files):
    """Lê e retorna uma lista de textos a partir de uma lista de arquivos."""
    texts = []
    for file_path in files:
        with open(file_path, "r", encoding="utf-8") as file:
            texts.append(file.read())
    return texts

def get_data_from_text_files(folder_name: str) -> pd.DataFrame:
    """Carrega os arquivos de texto a partir de uma pasta e embaralha os dados."""
    files = glob.glob(os.path.join("musicas", folder_name, "*.txt"))
    texts = get_text_list_from_files(files)
    df = pd.DataFrame({"lyric": texts})
    df = df.sample(frac=1).reset_index(drop=True)
    return df

# Carregando os dados de treino e validação (ajuste as pastas conforme a organização dos seus arquivos)
train_df = get_data_from_text_files("train")    
test_df = get_data_from_text_files("test")

print(f"Tamanho do DataFrame de Treino: {len(train_df)}")
print(f"Tamanho do DataFrame de Teste/Validação: {len(test_df)}")

def clean_text(text: str) -> str:
    """Aplica uma limpeza simples no texto: remoção de tags HTML, 
    conversão para minúsculas e remoção de pontuações."""
    text = text.lower()
    text = re.sub(r"<br\s*/?>", " ", text)
    # Remove pontuações definidas na expressão
    text = re.sub(r"[!#$%&'()*+,-./:;<=>?@\^_`{|}~]", "", text)
    return text

def tokenize(text: str) -> list:
    """Tokeniza o texto em palavras usando split básico."""
    text = clean_text(text)
    return text.split()

def build_vocab(texts: list, vocab_size: int = 30000):
    """Constroi o vocabulário a partir dos textos tokenizados. Se o vocabulário 
       exceder 'vocab_size', usa os tokens mais frequentes."""
    counter = Counter()
    for text in texts:
        tokens = tokenize(text)
        counter.update(tokens)
    # Seleciona os tokens mais frequentes (caso haja o parâmetro 'vocab_size')
    most_common = counter.most_common(vocab_size)
    vocab = [token for token, _ in most_common]
    
    # Cria mapeamentos: token -> ID e ID -> token (reserve o ID 0 para <unk>)
    token2id = {token: idx+1 for idx, token in enumerate(vocab)}
    token2id["<unk>"] = 0
    id2token = {idx: token for token, idx in token2id.items()}
    return token2id, id2token

# Constrói o vocabulário usando os dados de treino
all_train_texts = train_df['lyric'].tolist()
token2id, id2token = build_vocab(all_train_texts, vocab_size=30000)
print(f"Tamanho do vocabulário: {len(token2id)}")

def text_to_sequence(text: str, token2id: dict) -> list:
    """Converte um texto em uma sequência de IDs, usando '<unk>' para tokens desconhecidos."""
    tokens = tokenize(text)
    return [token2id.get(token, token2id["<unk>"]) for token in tokens]

# Converte os textos para sequências numéricas
train_sequences = [text_to_sequence(text, token2id) for text in all_train_texts]
test_sequences = [text_to_sequence(text, token2id) for text in test_df['lyric'].tolist()]


Tamanho do DataFrame de Treino: 596
Tamanho do DataFrame de Teste/Validação: 150
Tamanho do vocabulário: 6420


In [7]:
# ============================
# Passo 3: Definir os Componentes do HMM
# ============================

# Para utilizar um HMM, precisamos de:
# 1. Um número definido de estados ocultos: este é um hiperparâmetro a ser ajustado.
# 2. Probabilidades iniciais (pi): a probabilidade de cada estado iniciar uma sequência.
# 3. Matriz de transição (A): a probabilidade de transição entre estados.
# 4. Matriz de emissão (B): a probabilidade de cada estado emitir cada token do vocabulário.

# Aqui usamos a biblioteca hmmlearn para criar um modelo HMM multimonial, 
# onde as observações são os tokens (representados por seus IDs).

num_states = 10  # Defina o número de estados ocultos desejado

# Para treinar o HMM, precisamos de uma única sequência concatenada e 
# os tamanhos individuais de cada sequência (usado para separar as sequências)
train_concat = np.concatenate(train_sequences)
lengths = [len(seq) for seq in train_sequences]

# Converte para o formato exigido pela biblioteca (array com shape (n_samples, 1))
train_obs = np.array(train_concat).reshape(-1, 1)

# Inicializa o modelo HMM Multinomial
model = hmm.MultinomialHMM(n_components=num_states, n_iter=100, random_state=42, verbose=True)

# Define o número de possíveis símbolos (tamanho do vocabulário)
model.n_features = len(token2id)

# O treinamento (ajuste dos parâmetros do HMM via algoritmo Baum-Welch) 
# será realizado usando o método model.fit() passando as observações e os comprimentos.
# Exemplo de treinamento (opcional neste trecho):
# model.fit(train_obs, lengths)

print("Modelo HMM definido com:")
print(f" - Número de estados ocultos: {num_states}")
print(f" - Número de observações (tamanho do vocabulário): {model.n_features}")

# OBSERVAÇÃO:
# - O treinamento e ajustes dos parâmetros do HMM será realizado na etapa posterior.
# - Certifique-se de instalar a biblioteca hmmlearn (pip install hmmlearn) se ainda não estiver instalada.


MultinomialHMM has undergone major changes. The previous version was implementing a CategoricalHMM (a special case of MultinomialHMM). This new implementation follows the standard definition for a Multinomial distribution (e.g. as in https://en.wikipedia.org/wiki/Multinomial_distribution). See these issues for details:
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


Modelo HMM definido com:
 - Número de estados ocultos: 10
 - Número de observações (tamanho do vocabulário): 6420


In [ ]:
import numpy as np
import random

# ============================
# Passo 4: Estimar os Parâmetros do HMM
# ============================

print("Treinando o modelo HMM com o algoritmo Baum-Welch...")

# Crie o modelo definindo também n_features no construtor,
# onde n_features é o tamanho do vocabulário (número de símbolos discretos)
num_states = 10  # Número de estados ocultos
vocab_size = len(token2id)  # Tamanho do vocabulário

# Inicializa o modelo HMM Multinomial com n_components (estados)
model = hmm.MultinomialHMM(n_components=num_states, n_iter=100)

# Treinamento do HMM usando as observações concatenadas e os comprimentos individuais das sequências
model.fit(train_obs, lengths)
print("Treinamento concluído!\n")

# Exibindo alguns parâmetros aprendidos (opcional)
print("Probabilidades iniciais (startprob_):")
print(model.startprob_)
print("\nMatriz de transição (transmat_):")
print(model.transmat_)
print("\nMatriz de emissão (emissionprob_):")
print(model.emissionprob_)


# ============================
# Passo 5: Previsão de Tokens (Inferência)
# ============================

def predict_masked_token(sequence, mask_index, model, candidate_tokens=None):
    """
    Dada uma sequência de IDs de tokens com um token mascarado (placeholder),
    essa função substitui o token na posição 'mask_index' por cada candidato e
    calcula o log likelihood da sequência resultante. Retorna o token que maximiza
    a probabilidade da sequência.

    Args:
        sequence: lista de inteiros representando os tokens (com o token mascarado).
        mask_index: índice na sequência onde o token foi mascarado.
        model: modelo HMM treinado (hmm.MultinomialHMM).
        candidate_tokens: lista de IDs candidatos a substituir o token mascarado.
            Se None, utiliza todos os tokens do vocabulário (exceto o <unk> com ID 0).

    Retorna:
        best_token: o ID do token que maximiza o log likelihood.
        best_log_likelihood: o valor do log likelihood para a substituição escolhida.
    """
    # Se não definirmos candidatos, usamos todos os tokens válidos (exceto o ID 0, reservado para <unk>)
    if candidate_tokens is None:
        candidate_tokens = list(range(1, model.n_features))
    best_token = None
    best_log_likelihood = -np.inf  # Valor inicial para comparação
    for token in candidate_tokens:
        new_sequence = sequence.copy()
        new_sequence[mask_index] = token
        X = np.array(new_sequence).reshape(-1, 1)
        # Calcula o log likelihood da sequência com a substituição
        logL = model.score(X)
        if logL > best_log_likelihood:
            best_log_likelihood = logL
            best_token = token
    return best_token, best_log_likelihood

# Exemplo de uso:
# Seleciona uma sequência de teste e escolhe aleatoriamente uma posição para "mascarar"
test_sequence = test_sequences[0]  # Utilizando a primeira sequência de teste
print("\nSequência original (primeiros 20 tokens):", test_sequence[:20])

if len(test_sequence) > 0:
    # Seleciona um índice aleatório para mascarar
    mask_idx = random.randint(0, len(test_sequence) - 1)
    print(f"\nMascarando o token na posição {mask_idx} (token original: {test_sequence[mask_idx]})")
    
    original_token = test_sequence[mask_idx]
    masked_sequence = test_sequence.copy()
    # Usamos -1 como placeholder para o token mascarado
    masked_sequence[mask_idx] = -1

    # Prediz o token que preenche melhor essa posição
    predicted_token, ll = predict_masked_token(masked_sequence, mask_idx, model)
    print(f"\nToken predito (ID): {predicted_token} (Log Likelihood: {ll})")
    
    # Converte o ID do token predito para o token em texto (usando o dicionário id2token)
    token_text = id2token.get(predicted_token, "<desconhecido>")
    print("Token predito (texto):", token_text)


MultinomialHMM has undergone major changes. The previous version was implementing a CategoricalHMM (a special case of MultinomialHMM). This new implementation follows the standard definition for a Multinomial distribution (e.g. as in https://en.wikipedia.org/wiki/Multinomial_distribution). See these issues for details:
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


Treinando o modelo HMM com o algoritmo Baum-Welch...
Treinamento concluído!

Probabilidades iniciais (startprob_):
[3.75827206e-04 2.52932973e-05 8.59175053e-01 1.10004453e-01
 6.31905995e-04 2.10534761e-08 2.25761909e-02 2.81102365e-06
 8.25871809e-05 7.12585737e-03]

Matriz de transição (transmat_):
[[9.88338323e-04 3.65289340e-04 1.73500266e-02 3.04862330e-02
  8.11936526e-04 1.73011401e-03 9.29461058e-01 2.04179991e-03
  1.67652038e-02 5.99810008e-10]
 [2.13727610e-04 2.35801421e-03 1.36736510e-05 8.04622011e-02
  1.49978221e-01 2.85176618e-03 5.23810633e-03 7.58879022e-01
  1.58415166e-09 5.26600892e-06]
 [8.40592183e-02 8.16284182e-02 2.65710946e-01 7.89189640e-06
  3.21950455e-03 2.17411797e-07 1.75289602e-03 3.33343573e-03
  2.84225965e-03 5.57445212e-01]
 [1.05857532e-02 5.98123596e-05 1.58644890e-05 2.00007390e-07
  8.44347055e-06 4.69412245e-02 9.41765497e-01 9.71128041e-21
  4.87025239e-04 1.36179613e-04]
 [4.86182173e-01 8.37106956e-02 1.21342243e-02 5.00550941e-02
  8.464